In [188]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [211]:
df = pd.read_csv("new.csv")
df.head()

,bullying_words,type_bully
0,Get fucking real dude.,bully
1,She is as dirty as they come and that crook ...,bully
2,why did you fuck it up. I could do it all day...,bully
3,Dude they dont finish enclosing the fucking s...,bully
4,WTF are you talking about Men? No men thats n...,bully


In [212]:
# Processing data from commentsit being bullying_words
import spacy  

nlp = spacy.load("en_core_web_sm")
# function to preprocess
def preprocess(bullying_words):
    
    doc = nlp(bullying_words)
    
    filtered_tokens = []
    
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        # appending new values to the list
        filtered_tokens.append(token.lemma_)
    # converting alist to a string
    return " ".join(filtered_tokens)

preprocess("Get fucking real dude.")

'fucking real dude'

In [213]:
# checking if they balance
df.type_bully.value_counts()

type_bully
non-bully    12245
bully         7980
non             15
Name: count, dtype: int64

In [214]:
# under sampling taking random sample
min_sample = 7980
df_bully = df[df.type_bully == "bully"].sample(min_sample, random_state=40)
df_non_bully = df[df.type_bully == "non-bully"].sample(min_sample, random_state=40)

In [215]:
#it supply bunch of data frames into argument and add them row by row
df_balanced = pd.concat([df_bully, df_non_bully], axis=0)
df_balanced.type_bully.value_counts()

type_bully
bully        7980
non-bully    7980
Name: count, dtype: int64

In [216]:
# converting into numeric when it maps bully and non-bully
df_balanced['type_bully_num'] = df_balanced.type_bully.map(
    {'bully': 1, 'non-bully': 0
})
# checking it
df_balanced.head()

,bullying_words,type_bully,type_bully_num
4656,whats the operating system on it? i have windo...,bully,1
6129,Lol. Ur a bitch. I've done Hondurans and Chile...,bully,1
3506,"You didn't imagine it. It was the ""get away fr...",bully,1
7734,shits gay!,bully,1
4450,people hate to admit emotional decision making...,bully,1


In [217]:
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer

# # Assuming df is your DataFrame containing 'bullying_words' and 'type_bully' columns
# # Fill missing values with an empty string
# df['bullying_words'] = df['bullying_words'].fillna('')

# # Tokenization
# df['tokenized_text'] = df['bullying_words'].apply(word_tokenize)

# # Remove stop words
# stop_words = set(stopwords.words('english'))
# df['filtered_text'] = df['tokenized_text'].apply(lambda tokens: [word for word in tokens if word.lower() not in stop_words])

# # Convert text to TF-IDF vectors
# tfidf_vectorizer = TfidfVectorizer()
# X = tfidf_vectorizer.fit_transform(df['filtered_text'].apply(lambda tokens: ' '.join(tokens)))
# Y = df['type_bully']


In [218]:
X = df[df.columns[:-1]].values
Y = df[df.columns[1]].values

In [219]:
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

In [220]:
# training a model and the column type_bully_num is our target varviable
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming df is your DataFrame containing 'bullying_words' and 'type_bully' columns
# Fill missing values with an empty string
df['bullying_words'] = df['bullying_words'].fillna('')

# Tokenization
df['tokenized_text'] = df['bullying_words'].apply(word_tokenize)

# Remove stop words
stop_words = set(stopwords.words('english'))
df['filtered_text'] = df['tokenized_text'].apply(lambda tokens: [word for word in tokens if word.lower() not in stop_words])

# Convert text to TF-IDF vectors
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(df['filtered_text'].apply(lambda tokens: ' '.join(tokens)))
Y = df['type_bully']

# Split the data into training and testing sets(60% of data for training, 20% for validation, 20% for testing
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.4, random_state=0)
# it take unit of the data set and take it into 50/50
X_valid, X_test, Y_valid, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=0)

In [221]:
# building a TensorFlow model
model = tf.keras.Sequential([
    # layer it takes everything and output a value it has 16 layer
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [222]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
             loss = tf.keras.losses.BinaryCrossentropy(),
             metrics = ['accuracy'])

In [223]:
from sklearn.preprocessing import LabelEncoder
# Initialize LabelEncoder
label_encoder = LabelEncoder()
# Encode string labels to numerical values
Y_train_encoded = label_encoder.fit_transform(Y_train)
Y_test_encoded = label_encoder.transform(Y_test)

# Now you can use Y_train_encoded and Y_test_encoded for training and evaluation
model.evaluate(X_train, Y_train_encoded)

380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3763 - loss: 0.6976


[0.6975890398025513, 0.3754940629005432]

In [224]:
from sklearn.preprocessing import LabelEncoder
# Assuming Y_valid contains string labels
# Initialize LabelEncoder
label_encoder = LabelEncoder()
# Encode string labels to numerical values
Y_valid_encoded = label_encoder.fit_transform(Y_valid)
# Now you can use X_valid and Y_valid_encoded for evaluation
model.evaluate(X_valid, Y_valid_encoded)

127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3793 - loss: 0.6976


[0.6974648237228394, 0.38191699981689453]

In [225]:
# model.fit(X_train, Y_train, batch_size=16, epochs=20, validation_data=(X_valid, Y_valid)) 
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Encode string labels to numerical values for both training and validation sets
Y_train_encoded = label_encoder.fit_transform(Y_train)
Y_valid_encoded = label_encoder.transform(Y_valid)

# Now you can use X_train, Y_train_encoded, X_valid, and Y_valid_encoded for model training and evaluation
model.fit(X_train, Y_train_encoded, batch_size=16, epochs=20, validation_data=(X_valid, Y_valid_encoded))


Epoch 1/20
759/759 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.0086 - loss: -0.9640 - val_accuracy: 2.4704e-04 - val_loss: -19.2876
Epoch 2/20
759/759 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 6.1081e-04 - loss: -45.0983 - val_accuracy: 0.0020 - val_loss: -153.1577
Epoch 3/20
759/759 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.0018 - loss: -256.8987 - val_accuracy: 0.0022 - val_loss: -487.0391
Epoch 4/20
759/759 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.0036 - loss: -729.2184 - val_accuracy: 0.0025 - val_loss: -1070.7749
Epoch 5/20
759/759 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.0040 - loss: -1504.1384 - val_accuracy: 0.0025 - val_loss: -1917.8827
Epoch 6/20
759/759 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.0035 - loss: -2662.7646 - val_accuracy: 0.0027 - val_loss: -3051.5583
Epoch 7/20
759/759 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.0049 - loss: -3934.1096 - val_accuracy: 0.0030 - val_loss: -4514.7900
Epoch 8/20
759/759 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step

In [232]:
from sklearn.metrics import classification_report

# Assuming Y_test and Y_pred are your true and predicted labels
# Convert Y_test to numeric labels if they are strings
if Y_test.dtype == 'object':
    label_mapping = {label: idx for idx, label in enumerate(np.unique(Y_test))}
    Y_test = np.array([label_mapping[label] for label in Y_test])

# Convert Y_pred to numeric labels if they are probabilities
if Y_pred.shape[1] > 1:  # If Y_pred is probabilities, convert to class predictions
    Y_pred = np.argmax(Y_pred, axis=1)

# Generate the classification report with zero_division=1
report = classification_report(Y_test, Y_pred, zero_division=1)
print(report)


              precision    recall  f1-score   support

           0       1.00      0.01      0.02      1549
           1       0.00      1.00      0.00         2
           2       1.00      0.00      0.00      2497

    accuracy                           0.00      4048
   macro avg       0.67      0.34      0.01      4048
weighted avg       1.00      0.00      0.01      4048



In [235]:
from sklearn import metrics
metrics.accuracy_score(Y_test, Y_pred)

0.003952569169960474

In [203]:
Y_train.value_counts()

type_bully
non-bully    7324
bully        4808
non            12
Name: count, dtype: int64